# Test conversion on Niel's alpha-cyldextrin bound to 1-butylamine[+1] file!
```
/data/nhenriksen/projects/cds/wat6/bgbg-tip3p/a-bam-p/a00
```

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import subprocess as sp

from openeye.oechem import *
from openforcefield.typing.engines.smirnoff import *
from openforcefield.utils import mergeStructure
import parmed as pmd

from utils import create_pdb_with_conect, prune_conect
from utils import split_topology, create_host_guest_topology
from utils import create_host_mol2
from utils import load_mol2, check_unique_atom_names, load_pdb
from utils import extract_water_and_ions, create_water_and_ions_parameters
from utils import extract_dummy_atoms, create_dummy_atom_parameters
from utils import map_residues, map_atoms
from utils import copy_box_vectors
from utils import rewrite_restraints_file, rewrite_amber_input_file

# Create a standards-compliant PDB...

In [2]:
create_pdb_with_conect(solvated_pdb='original/a-bam-p/full.crds', 
                       amber_prmtop='original/a-bam-p/full.topo', 
                       output_pdb='generated/a-bam-p/full.pdb')
prune_conect(input_pdb='full.pdb', 
             output_pdb='full_conect.pdb', 
             path='generated/a-bam-p/')

PDB file written by cpptraj.
First water residue = 160
Found first water CONECT entry at line = 8322


# Split the PDB into components and generate a topology for each of them...

In [3]:
components = split_topology(file_name='generated/a-bam-p/full.pdb')
hg_topology = create_host_guest_topology(components, host_resname='MGO', guest_resname='BAM')

# Load each topology component (from `mol2`) into separate `OEMol`s...

In [4]:
host = load_mol2(filename='original/a-bam-p/MGO.mol2', 
                 name='MGO', 
                 add_tripos=True)
guest = load_mol2(filename='original/a-bam-p/bam.mol2', 
                  name='BAM', 
                  add_tripos=False)
check_unique_atom_names(host)
check_unique_atom_names(guest)
molecules = [host, guest]

21 atoms in structure, 21 unique atom names.
17 atoms in structure, 17 unique atom names.


# Polymerize the host molecule and make sure there are *unique* atom types for every atom in the host...

In [5]:
create_host_mol2('generated/a-bam-p/full.pdb', 'original/a-bam-p/full.topo', 'MGO', 'generated/a-bam-p/MGO.mol2')

MOL2 file written by cpptraj.


In [6]:
host = load_mol2(filename='generated/a-bam-p/MGO.mol2', 
                 name='MGO', 
                 add_tripos=True)
guest = load_mol2(filename='original/a-bam-p/bam.mol2', 
                  name='BAM', 
                  add_tripos=False)
check_unique_atom_names(host)
check_unique_atom_names(guest)
molecules = [host, guest]

126 atoms in structure, 126 unique atom names.
17 atoms in structure, 17 unique atom names.


# Parameterize the host and guest with SMIRNOFF99Frosst...

In [7]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml') 
system = ff.createSystem(hg_topology.topology, molecules,
                         nonbondedCutoff=1.1*unit.nanometer, 
                         ewaldErrorTolerance=1e-4
                         )

# Convert the parameterized system in a ParmEd structure -- now with the SMIRNOFF99Frosst parameters...

In [8]:
hg_structure = pmd.openmm.topsystem.load_topology(hg_topology.topology, system, hg_topology.positions)

# ...and save this as an AMBER-format `prmtop` and `inpcrd`...

In [9]:
try:
    hg_structure.save('generated/a-bam-p/hg.prmtop')
except OSError:
    print('Check if the file already exists...')

Check if the file already exists...


In [10]:
try:
    hg_structure.save('generated/a-bam-p/hg.inpcrd')
except OSError:
    print('Check if the file already exists...')

Check if the file already exists...


# Use the original AMBER files to extract the waters and ions (and dummy atoms) and then parameterize those with GAFF or something else...

In [11]:
extract_water_and_ions(amber_prmtop='original/a-bam-p/full.topo',
                      amber_inpcrd='original/a-bam-p/full.crds',
                      host_residue=':MGO',
                      guest_residue=':BAM',
                      dummy=None,
                      output_pdb='generated/a-bam-p/water_ions.pdb')

Water and ion PDB file written by cpptraj.


In [12]:
create_water_and_ions_parameters(input_pdb='water_ions.pdb',
                                output_prmtop='water_ions.prmtop',
                                output_inpcrd='water_ions.inpcrd',
                                dummy_atoms=True,
                                path='generated/a-bam-p/')

Writing dummy atom `frcmod`.
Writing dummy atom `mol2`.
Water and ion parameters and coordinates written by tleap.


# Load those parameters and coordinates into a ParmEd structure... and merge with the host-guest ParmEd structure...

In [13]:
water_and_ions = pmd.amber.AmberParm('generated/a-bam-p/water_ions.prmtop', xyz='generated/a-bam-p/water_ions.inpcrd')

In [14]:
merged = mergeStructure(hg_structure, water_and_ions)

In [15]:
try:
    merged.save('generated/a-bam-p/solvated_smirnoff.prmtop')
    merged.save('generated/a-bam-p/solvated_smirnoff.inpcrd')
except:
    print('Check if file exists...')

Check if file exists...


# Now, because we lumped the dummy atoms with the water and ions, they are no longer at the beginning of the coordinate file (if we extract the dummy atoms, then [ParmEd can't merge](https://github.com/ParmEd/ParmEd/issues/952) the structures). Thus, we need to remap all restraints to new atom and residue indices...

However, we need to do the *atom* mapping on a `mol2` file and the *residue* mapping on a `pdb` file -- where we can specifically choose to *not* ignore dummy atoms -- otherwise the graph is not isomorphic...

In [16]:
reference = pmd.load_file('original/a-bam-p/full.topo', 'original/a-bam-p/full.crds')
try:
    reference.save('generated/a-bam-p/reference.pdb')
    reference.save('generated/a-bam-p/reference.mol2')
except OSError:
    print('Check if file exists...')
target = pmd.load_file('generated/a-bam-p/solvated_smirnoff.prmtop', 'generated/a-bam-p/solvated_smirnoff.inpcrd')
try:
    target.save('generated/a-bam-p/target.pdb')
    target.save('generated/a-bam-p/target.mol2')
except OSError:
    print('Check if file exists...')

Check if file exists...
Check if file exists...


In [17]:
reference_mol = load_mol2('generated/a-bam-p/reference.mol2')
target_mol = load_mol2('generated/a-bam-p/target.mol2')

In [18]:
atom_mapping = map_atoms(reference_mol, target_mol)

Determining mapping...
Reference → Target


Before I do that, make sure the mapping doesn't have `None`...

In [19]:
atom_mapping

{3: 0,
 4: 1,
 5: 2,
 6: 3,
 7: 4,
 8: 5,
 9: 6,
 10: 7,
 11: 8,
 12: 9,
 13: 10,
 14: 11,
 15: 12,
 16: 13,
 17: 14,
 18: 15,
 19: 16,
 20: 17,
 21: 18,
 22: 19,
 23: 20,
 35: 32,
 31: 28,
 27: 24,
 24: 21,
 25: 22,
 26: 23,
 28: 25,
 29: 26,
 30: 27,
 32: 29,
 33: 30,
 34: 31,
 36: 33,
 37: 34,
 38: 35,
 39: 36,
 40: 37,
 41: 38,
 42: 39,
 43: 40,
 44: 41,
 56: 53,
 52: 49,
 48: 45,
 45: 42,
 46: 43,
 47: 44,
 49: 46,
 50: 47,
 51: 48,
 53: 50,
 54: 51,
 55: 52,
 57: 54,
 58: 55,
 59: 56,
 60: 57,
 61: 58,
 62: 59,
 63: 60,
 64: 61,
 65: 62,
 77: 74,
 73: 70,
 69: 66,
 66: 63,
 67: 64,
 68: 65,
 70: 67,
 71: 68,
 72: 69,
 74: 71,
 75: 72,
 76: 73,
 78: 75,
 79: 76,
 80: 77,
 81: 78,
 82: 79,
 83: 80,
 84: 81,
 85: 82,
 86: 83,
 98: 95,
 94: 91,
 90: 87,
 87: 84,
 88: 85,
 89: 86,
 91: 88,
 92: 89,
 93: 90,
 95: 92,
 96: 93,
 97: 94,
 99: 96,
 100: 97,
 101: 98,
 102: 99,
 103: 100,
 104: 101,
 105: 102,
 106: 103,
 107: 104,
 110: 107,
 108: 105,
 109: 106,
 111: 108,
 112: 109,
 113

In [33]:
all(atom_mapping.values() == None)

TypeError: 'bool' object is not iterable

In [30]:
atom_mapping.viewvalues()

AttributeError: 'dict' object has no attribute 'viewvalues'

In [ ]:
# reference_mol = load_pdb('generated/a-bam-p/reference.pdb')
# target_mol = load_pdb('generated/a-bam-p/target.pdb')

In [ ]:
# residue_mapping = map_residues(atom_mapping, reference_mol, target_mol)

# Now let's copy over the minimization input file, and replace the residue indices of the positional restraints...

In [ ]:
rewrite_amber_input_file(reference_input='original/a-bam-p/mini.in',
                        target_input='generated/a-bam-p/mini.in',
                        reference_to_target_mapping=residue_mapping)

# Now let's copy over the restraint file, and replace the atom indices...

In [ ]:
rewrite_restraints_file(reference_restraints='original/a-bam-p/disang',
                       target_restraints='generated/a-bam-p/disang',
                       reference_to_target_mapping=atom_mapping)

# Finally, copy over the box dimensions and angles, and we should be good to go...

In [ ]:
copy_box_vectors(input_inpcrd='original/a-bam-p/full.crds',
                output_inpcrd='generated/a-bam-p/solvated_smirnoff.inpcrd')